#### this notebook is part of the documentation on my HPA approach  
    -> main notebook: https://www.kaggle.com/philipjamessullivan/0-hpa-approach-summary
    
# 2: combine image channels
 ## RESULTS:
**dataset name:** 20 datasets with names "hpa-composite-images-{}-of-20" (linked to this notebook)  
        --> dataset is in 20 parts because of kaggle errors with saving everything as one dataset  
**file type:** png files  
**contents:** images from HPA dataset combined from RGBY to RGB  

In [ ]:
#constants
IMG_FOLDER_PATH="../input/hpa-single-cell-image-classification/train/"
CSV_FILE_PATH="../input/hpa-single-cell-image-classification/train.csv"

In [ ]:
#load HPA dataset
import pandas as pd
CSV_FILE_PATH="../input/hpa-single-cell-image-classification/train.csv"
id_labels_array=pd.read_csv(CSV_FILE_PATH)
id_array=(id_labels_array["ID"]).tolist()

In [ ]:
#function to combine rgby to rgb
#source:https://www.kaggle.com/kwentar/visualization-examples-of-each-class-in-rgb#Load-data:
import numpy as np
def rgby_to_rgb(r,g,b,y):
    image_width,image_height=r.size
    rgb_image = np.zeros(shape=(image_height, image_width, 3), dtype=np.float)
    yellow = np.array(y)
    # yellow is red + green
    rgb_image[:, :, 0] += yellow/2   
    rgb_image[:, :, 1] += yellow/2
    # loop for R,G and B channels
    for index, channel in enumerate([r,g,b]):
        current_image = channel
        rgb_image[:, :, index] += current_image
    # Normalize image
    rgb_image = rgb_image / rgb_image.max() * 255
    return rgb_image.astype(np.uint8)
#function to get rgb image from only img_id
from PIL import Image
def imgid_to_rgb(img_id):
    r=Image.open(IMG_FOLDER_PATH+img_id+"_red.png")
    g=Image.open(IMG_FOLDER_PATH+img_id+"_green.png")
    b=Image.open(IMG_FOLDER_PATH+img_id+"_blue.png")
    y=Image.open(IMG_FOLDER_PATH+img_id+"_yellow.png")
    rgb=rgby_to_rgb(r,g,b,y)
    return rgb

In [ ]:
#function to convert all images to rgb images
#exceeds 9h kaggle runtime so this must be run locally or separated into 20 parts and run in parallel in separate notebooks
#these files have been saved in the datasets named hpa-composite-images-x-of-20
from PIL import Image
import numpy
from tqdm import tqdm

COMPOSITE_IMG_PATH="./composites/"
import os
if not os.path.exists(COMPOSITE_IMG_PATH):
    os.makedirs(COMPOSITE_IMG_PATH)

for img_id in tqdm(id_array[:10]):  ##ONLY RUN 10 FOR DEMONSTRATION!
    img_rgb = Image.fromarray(imgid_to_rgb(img_id))
    img_rgb.save(COMPOSITE_IMG_PATH+img_id+".png")
    

In [ ]:
#save to zip
import shutil
zip_name = 'composites'
directory_name = 'composites'

shutil.make_archive(zip_name, 'zip', directory_name)

In [ ]:
#remove original folder
!rm -r composites